In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [2]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [6]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(5)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [7]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [8]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [9]:
course_data_records = course_data_df.to_dict('records')
course_data_records[0]

{'Course_Code__c': 'BC-DATAVIZ',
 'Course_Name__c': 'Data Visualization and Analytics',
 'Credit_Hours__c': 12,
 'Boot_Camp_Course__c': 1}

In [10]:
#Deleting courses records
course_records = sf.query("SELECT Name FROM Course__c")
recs_to_delete = [{'Name': r['Name']} for r in course_records['records']]
recs_to_delete

[{'Name': 'a004x000003WHR4'},
 {'Name': 'a004x000003WHRO'},
 {'Name': 'a004x000003WHRJ'},
 {'Name': 'a004x000003WHQz'},
 {'Name': 'a004x000003WHR9'},
 {'Name': 'a004x000003WHQu'},
 {'Name': 'a004x000003WHRE'},
 {'Name': 'a004x000003WAAs'}]

### Deleting from `Course` Record

In [11]:
for rec in recs_to_delete:
    try:
        sf.Course__c.delete(rec['Name'])
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/a004x000003WAAs. Response content: [{'message': 'Your attempt to delete a004x000003WAAs could not be completed because it is associated with the following class participants.: a094x000000ascR, a094x000000ash1, a094x000000aslI, a094x000000aslR, ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ...\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]


In [12]:
# Example of insurting in Bulk 
# sf.bulk.Course__c.insert(course_data_records)

### Inserting Data into `course` Record in SF

In [13]:

for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/. Response content: [{'message': 'duplicate value found: Course_Code__c duplicates value on record with id: a004x000003WAAs', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]


### Create `Course` Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [14]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Name, Course_Code__c FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003WJQL', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003WJQB', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003WJPr', 'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Course__c': 'a004x000003WJQ6', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003WJPw', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003WJQG', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003WJQ1', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003WAAs', 'Course_Code__c': 'BC-DATAVIZ'}]

In [15]:
#Convert into DataFrame 
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003WJQL,CIS-435
1,a004x000003WJQB,CIS-438
2,a004x000003WJPr,BC-WEBDEV
3,a004x000003WJQ6,CIS-405
4,a004x000003WJPw,BC-UIUX
5,a004x000003WJQG,CIS-430
6,a004x000003WJQ1,CIS-349
7,a004x000003WAAs,BC-DATAVIZ


### Query the `Class` table from MySQL


In [16]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [17]:

class_records = class_data_df.to_dict(orient='records')
class_records

[{'CourseCode': 'BC-DATAVIZ',
  'ID_Class': 1,
  'ID_Course': 1,
  'Section': 'GWU-ARL-DATA-PT-09-0',
  'StartDate': datetime.date(2020, 9, 16),
  'EndDate': datetime.date(2020, 3, 14)},
 {'CourseCode': 'BC-DATAVIZ',
  'ID_Class': 2,
  'ID_Course': 1,
  'Section': 'GWDC201805DATA3',
  'StartDate': datetime.date(2018, 5, 15),
  'EndDate': datetime.date(2018, 11, 8)},
 {'CourseCode': 'BC-WEBDEV',
  'ID_Class': 3,
  'ID_Course': 2,
  'Section': 'GWARL201905WEB3',
  'StartDate': datetime.date(2019, 5, 14),
  'EndDate': datetime.date(2019, 11, 7)},
 {'CourseCode': 'BC-UIUX',
  'ID_Class': 4,
  'ID_Course': 3,
  'Section': 'GWARL201905UIUX3',
  'StartDate': datetime.date(2019, 5, 14),
  'EndDate': datetime.date(2019, 11, 7)}]

In [18]:
#rename columns from class table
class_data_df.rename(columns={
    'ID_Course': 'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


### Join the `Class` DataFrame with the `Course` lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [19]:
class_xwalk_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_xwalk_df.head()


#class_data_df
class_xwalk_df['Start_Date__c'] = class_xwalk_df['Start_Date__c'].astype(str)
class_xwalk_df['End_Date__c'] = class_xwalk_df['End_Date__c'].astype(str)
class_xwalk_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003WAAs
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003WAAs
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003WJPr
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003WJPw


In [20]:
class_xwalk_recs = class_xwalk_df.to_dict(orient='records')
class_xwalk_recs[0]

{'Course_Code__c': 'BC-DATAVIZ',
 'Section__c': 'GWU-ARL-DATA-PT-09-0',
 'Start_Date__c': '2020-09-16',
 'End_Date__c': '2020-03-14',
 'ID_Course__c': 'a004x000003WAAs'}

### Deleting from `Class` Record

In [21]:
class_records = sf.query("SELECT Name FROM Class__c")
recs_to_delete = [{'Name': r['Name']} for r in class_records['records']]
recs_to_delete

[{'Name': 'a014x00000A3HjQ'},
 {'Name': 'a014x00000A3dAX'},
 {'Name': 'a014x00000A3dAc'},
 {'Name': 'a014x00000A3DbZ'}]

In [22]:
for rec in recs_to_delete:
    try:
        sf.class__c.delete(rec['Name'])
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/class__c/a014x00000A3HjQ. Response content: [{'message': 'Your attempt to delete a014x00000A3HjQ could not be completed because it is associated with the following class participants.: a094x000000ascR, a094x000000ash1, a094x000000aslI, a094x000000aso1, ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ...\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/class__c/a014x00000A3dAX. Response content: [{'message': 'Your attempt to delete a014x00000A3dAX could not be completed because it is associated with the following class participants.: a094x000000asrA, a094x000000asrT, a094x000000asri, a094x000000asrx, ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ...\n', 'errorCod

## Insert `Class` Records into Salesforce

In [23]:
for rec in class_xwalk_recs:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': rec['Start_Date__c'],
        'End_Date__c': rec['End_Date__c'],
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Prepare `Student` Data

In [24]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [25]:
student_data_df.rename(columns={
    'StudentID':'Student_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Birth_Date__c',
    'Gender':'Gender__c',
}, inplace=True)
student_data_df.head()

,ID_Student,Student_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [26]:
student_data_df = student_data_df[['Student_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c', 'Gender__c']]
student_data_df.head()

,Student_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [27]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'Student_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mariette',
  'Birth_Date__c': None,
  'Gender_

### Deleting from `student` Record

In [28]:
student_records = sf.query("SELECT Name FROM Student__c")
recs_to_delete = [{'Name': r['Name']} for r in student_records['records']]
recs_to_delete

[{'Name': 'a064x000004SqVH'},
 {'Name': 'a064x000004SqTu'},
 {'Name': 'a064x000004SqVg'},
 {'Name': 'a064x000004SqUE'},
 {'Name': 'a064x000004SqU4'},
 {'Name': 'a064x000004SqVR'},
 {'Name': 'a064x000004SqTk'},
 {'Name': 'a064x000004SqVW'},
 {'Name': 'a064x000004SqUn'},
 {'Name': 'a064x000004SqVv'},
 {'Name': 'a064x000004SsjY'},
 {'Name': 'a064x000004Ssk7'},
 {'Name': 'a064x000004SqW0'},
 {'Name': 'a064x000004SqU9'},
 {'Name': 'a064x000004SqTf'},
 {'Name': 'a064x000004SqTp'},
 {'Name': 'a064x000004SqUO'},
 {'Name': 'a064x000004SqTz'},
 {'Name': 'a064x000004SqUi'},
 {'Name': 'a064x000004SqVb'},
 {'Name': 'a064x000004SqUY'},
 {'Name': 'a064x000004SqUT'},
 {'Name': 'a064x000004SqUJ'},
 {'Name': 'a064x000004SqVl'},
 {'Name': 'a064x000004SqV2'},
 {'Name': 'a064x000004SqV7'},
 {'Name': 'a064x000004SqVq'},
 {'Name': 'a064x000004SqUs'},
 {'Name': 'a064x000004SqVM'},
 {'Name': 'a064x000004SqUx'}]

In [29]:
for rec in recs_to_delete:
    try:
        sf.Student__c.delete(rec['Name'])
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SqVH. Response content: [{'message': 'Your attempt to delete a064x000004SqVH could not be completed because it is associated with the following class participants.: a094x000000ascR, a094x000000asqR, a094x000000asvB\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SqTu. Response content: [{'message': 'Your attempt to delete a064x000004SqTu could not be completed because it is associated with the following class participants.: a094x000000assM, a094x000000assR, a094x000000assW\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SqVg. Response content: [{'message': 'Your attempt to delete a064x000004SqVg could not be completed because it is a

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SqV2. Response content: [{'message': 'Your attempt to delete a064x000004SqV2 could not be completed because it is associated with the following class participants.: a094x000000asum, a094x000000asur, a094x000000asuw\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SqV7. Response content: [{'message': 'Your attempt to delete a064x000004SqV7 could not be completed because it is associated with the following class participants.: a094x000000asqH, a094x000000asv1, a094x000000asv6\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SqVq. Response content: [{'message': 'Your attempt to delete a064x000004SqVq could not be completed because it is a

### Insert `student` Records into Salesforce

In [30]:
for rec in student_data_records:

    record = {
        'Student_ID__c': rec['Student_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birth_Date__c': rec['Birth_Date__c'],
        'Gender__c': rec['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SqTf', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SqTk', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SqTp', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value fo

### Prepare The Staff Data

In [33]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [63]:
#Rename Staff Data
staff_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
}, inplace=True)



In [64]:
staff_data_df = staff_data_df[['Employee_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c']]
staff_data_df.head()

,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [65]:
#pull staff data records
staff_data_records = staff_data_df.to_dict('records')
staff_data_records


[{'Employee_ID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'Employee_ID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'Employee_ID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'Employee_ID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

### Deleting `Staff` record

In [66]:
staff_records = sf.query("SELECT Id FROM Staff__c")
recs_to_delete = [{'Id': r['Id']} for r in staff_records['records']]
recs_to_delete

sf.bulk.Staff__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a054x0000010rsFAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a054x0000010rsKAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a054x0000010rsPAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a054x0000010rsUAAQ', 'errors': []}]

In [67]:
for rec in recs_to_delete:
    try:
        sf.Staff__c.delete(rec['Name'])
    except Exception as e:
        print(e)

'Name'
'Name'
'Name'
'Name'


### Instering Data Insto `Staff` Table in SF

In [39]:
for rec in staff_data_records:

    record = {
        'Employee_ID__c': rec['Employee_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birth_Date__c': str(rec['Birth_Date__c']),
        
    }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)


## Query `Student` Records from MySQL

In [40]:
query = '''
    SELECT 
        StudentID AS Student_ID__c,
        LastName AS Last_Name__c,
        FirstName AS First_Name__c,
        MiddleName AS Middle_Name__c,
        BirthDate AS Birth_Date__c,
        Gender AS Gender__c
    FROM
        student
'''

student_df = pd.read_sql(query, conn)


student_dict = student_df.to_dict(orient='records')

student_dict

[{'Student_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mariette',
  'Birth_Date__c': None,
  'Gender_

In [41]:
try:
    for rec in student_dict:
        sf.Student__c.create(rec)
except Exception as e:
    print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SqTf', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]


In [42]:
for rec in recs_to_delete:
    try:
        sf.Class_Participant__c.delete(rec['Name'])
    except Exception as e:
        print(e)

'Name'
'Name'
'Name'
'Name'


### Create `Class` Lookup and Load `Classparticipants` Data into SF

In [43]:

sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a004x000003WJQL,CIS-435
1,a004x000003WJQB,CIS-438
2,a004x000003WJPr,BC-WEBDEV
3,a004x000003WJQ6,CIS-405
4,a004x000003WJPw,BC-UIUX


In [68]:
#create class lookup table 
sf_class_records = []

# The `Name` column in the primary key in Salesforce objects
sf_class_data = sf.query_all_iter("SELECT Section__c, Name FROM Class__c")
for row in sf_class_data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'Section__c': row['Section__c']
    }
    sf_class_records.append(rec)

#convert to dataframe
class_lookup_df = pd.DataFrame(sf_class_records)
class_lookup_df




,ID_Class__c,Section__c
0,a014x00000A3HjQ,GWU-ARL-DATA-PT-09-0
1,a014x00000A3dAX,GWU-ARL-DATA-PT-09-0
2,a014x00000A3tU4,GWU-ARL-DATA-PT-09-0
3,a014x00000A3tU9,GWDC201805DATA3
4,a014x00000A3tUE,GWARL201905WEB3
5,a014x00000A3tUJ,GWARL201905UIUX3
6,a014x00000A3DbZ,GWU-ARL-DATA-PT-09-0


In [45]:
sf_student_recrds = []

# The `Name` column in the primary key in Salesforce objects
sf_student_data = sf.query_all_iter("SELECT Student_ID__c, Name FROM Student__c")
for row in sf_student_data:
    rec = {
        'ID_Student__c': row['Name'], # this is a critical line of code
        'Student_ID__c': row['Student_ID__c']
    }
    sf_student_recrds.append(rec)
    
#convert to dataframe
student_lookup_df = pd.DataFrame(sf_student_recrds)
student_lookup_df.head()

,ID_Student__c,Student_ID__c
0,a064x000004SqVH,25005250
1,a064x000004SqTu,25002589
2,a064x000004SqVg,25006762
3,a064x000004SqUE,25003778
4,a064x000004SqU4,25006014


In [46]:
#query the class Partcipant, class and student
query = '''

    SELECT
        s.StudentID AS Student_ID__c,
        cl.Section AS Section__c,
        cp.StartDate AS Start_Date__c,
        cp.EndDate AS End_Date__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

class_particiant_df = pd.read_sql(query, conn)
class_particiant_df.head()

,Student_ID__c,Section__c,Start_Date__c,End_Date__c
0,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [69]:
class_participant_df= pd.merge(class_particiant_df, class_lookup_df, on=['Section__c'])
class_participant_df.head()


,Student_ID__c,Section__c,Start_Date__c,End_Date__c,ID_Class__c
0,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A3HjQ
1,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A3dAX
2,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A3tU4
3,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A3DbZ
4,25003514,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A3HjQ


In [70]:
class_participant_load = pd.merge(class_participant_df, student_lookup_df, how='left', on='Student_ID__c')
class_participant_load = class_participant_load[['ID_Student__c', 'ID_Class__c', 'Start_Date__c', 'End_Date__c' ]]
class_participant_load.head()


,ID_Student__c,ID_Class__c,Start_Date__c,End_Date__c
0,a064x000004SqTf,a014x00000A3HjQ,2020-09-16,None
1,a064x000004SqTf,a014x00000A3dAX,2020-09-16,None
2,a064x000004SqTf,a014x00000A3tU4,2020-09-16,None
3,a064x000004SqTf,a014x00000A3DbZ,2020-09-16,None
4,a064x000004SqTk,a014x00000A3HjQ,2020-09-16,None


In [71]:
clasparticipant_records = class_participant_load.to_dict('records')
clasparticipant_records[0]

{'ID_Student__c': 'a064x000004SqTf',
 'ID_Class__c': 'a014x00000A3HjQ',
 'Start_Date__c': datetime.date(2020, 9, 16),
 'End_Date__c': None}

In [76]:
sf_class_df = pd.DataFrame(sf_class_records)

#sf_class_df['ID_Course__c'] = sf_class_df['ID_Course__c'].apply(lambda id_course: id_course[:-3])

sf_class_df.head()


,ID_Class__c,Section__c
0,a014x00000A3HjQ,GWU-ARL-DATA-PT-09-0
1,a014x00000A3dAX,GWU-ARL-DATA-PT-09-0
2,a014x00000A3tU4,GWU-ARL-DATA-PT-09-0
3,a014x00000A3tU9,GWDC201805DATA3
4,a014x00000A3tUE,GWARL201905WEB3


### Deleting `classparticipant` records

In [73]:
class_participant_records_del = sf.query("SELECT Name FROM Class_Participant__c")
recs_to_delete = [{'Name': r['Name']} for r in class_participant_records_del['records']]
recs_to_delete



[]

In [48]:
for rec in recs_to_delete:
    try:
        sf.Class_Participant__c.delete(rec['Name'])
    except Exception as e:
        print(e)

In [74]:
#inserting records 
for rec in clasparticipant_records:
    #taking a key out of dictionary putting into new dic format
    record = {}
    record['ID_Class__c'] = rec['ID_Class__c']
    record['ID_Student__c'] = rec['ID_Student__c']
    if rec['Start_Date__c'] != None:
        record['Start_Date__c'] = str(rec['Start_Date__c'])
    if rec['End_Date__c'] != None:
        record['End_Date__c'] = str(rec['End_Date__c'])
    
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)

### preparing `Staffassignment` and loading data

In [50]:
sf_staff_records = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
sf_staff_data = sf.query_all_iter("SELECT Employee_ID__c, Name FROM Staff__c")
for row in sf_staff_data:
    rec = {
        'ID_Staff__c': row['Name'], # this is a critical line of code
        'Employee_ID__c': row['Employee_ID__c']
    }
    sf_staff_records.append(rec)
    
#convert to dataframe
staff_lookup_df = pd.DataFrame(sf_staff_records)
staff_lookup_df

,ID_Staff__c,Employee_ID__c
0,a054x0000010rsF,000184220
1,a054x0000010rsP,000160655
2,a054x0000010rsU,000159108
3,a054x0000010rsK,000130109


In [51]:
query = '''
    SELECT 
    
        sa.StartDate,
        sa.EndDate,
        sa.Role,
        s.EmployeeID,
        c.Section
        
    FROM staffassignment sa
    INNER JOIN staff s
    ON s.ID_Staff = sa.ID_Staff
    INNER JOIN class c
    ON c.ID_Class = sa.ID_Class
'''

staff_assignment_data_df = pd.read_sql(query, conn)
staff_assignment_data_df.head()

,StartDate,EndDate,Role,EmployeeID,Section
0,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0
1,2020-09-16,None,Teacher Assistant,000159108,GWU-ARL-DATA-PT-09-0
2,2020-09-16,None,Instructor,000160655,GWU-ARL-DATA-PT-09-0


In [52]:
#rename columns from staff assign table
staff_assignment_data_df.rename(columns={
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'Role':'Role__c',
    'EmployeeID':'Employee_ID__c',
    'Section':'Section__c'
}, inplace=True)

staff_assignment_data_df.head()

,Start_Date__c,End_Date__c,Role__c,Employee_ID__c,Section__c
0,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0
1,2020-09-16,None,Teacher Assistant,000159108,GWU-ARL-DATA-PT-09-0
2,2020-09-16,None,Instructor,000160655,GWU-ARL-DATA-PT-09-0


In [53]:
sa_load_df = pd.merge(staff_assignment_data_df, class_lookup_df, how='left', on='Section__c')


sa_load_df.head()

,Start_Date__c,End_Date__c,Role__c,Employee_ID__c,Section__c,ID_Class__c
0,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3HjQ
1,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3dAX
2,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3tU4
3,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3DbZ
4,2020-09-16,None,Teacher Assistant,000159108,GWU-ARL-DATA-PT-09-0,a014x00000A3HjQ


In [78]:
staffassignment_load_df = pd.merge(sa_load_df, staff_lookup_df, how='left', on='Employee_ID__c')
staffassignment_load_df.head()
                                   

,Start_Date__c,End_Date__c,Role__c,Employee_ID__c,Section__c,ID_Class__c,ID_Staff__c
0,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3HjQ,a054x0000010rsF
1,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3dAX,a054x0000010rsF
2,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3tU4,a054x0000010rsF
3,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3DbZ,a054x0000010rsF
4,2020-09-16,None,Teacher Assistant,000159108,GWU-ARL-DATA-PT-09-0,a014x00000A3HjQ,a054x0000010rsU


In [58]:
staffassignment_load_df['Start_Date__c'] = staffassignment_load_df['Start_Date__c'].astype(str)
staffassignment_load_df['End_Date__c'] = staffassignment_load_df['End_Date__c'].astype(str)
staffassignment_load_df.head()

,Start_Date__c,End_Date__c,Role__c,Employee_ID__c,Section__c,ID_Class__c,ID_Staff__c
0,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3HjQ,a054x0000010rsF
1,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3dAX,a054x0000010rsF
2,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3tU4,a054x0000010rsF
3,2020-09-16,None,Teacher Assistant,000184220,GWU-ARL-DATA-PT-09-0,a014x00000A3DbZ,a054x0000010rsF
4,2020-09-16,None,Teacher Assistant,000159108,GWU-ARL-DATA-PT-09-0,a014x00000A3HjQ,a054x0000010rsU


In [59]:
staffassignments_records = staffassignment_load_df.to_dict('records')
staffassignments_records[0]



{'Start_Date__c': '2020-09-16',
 'End_Date__c': 'None',
 'Role__c': 'Teacher Assistant',
 'Employee_ID__c': '000184220',
 'Section__c': 'GWU-ARL-DATA-PT-09-0',
 'ID_Class__c': 'a014x00000A3HjQ',
 'ID_Staff__c': 'a054x0000010rsF'}

In [60]:
#delets staff assignment data
class_records = sf.query("SELECT Id FROM Staff_Assignment__c")
recs_to_delete = [{'Name': r['Name']} for r in class_records['records']]
recs_to_delete

sf.bulk.Staff_Assignment__c.delete(recs_to_delete)

[]

In [ ]:
for rec in staffassignments_records:
    try:
        sf.Staff_Assignment__c.create(rec)
    except Exception as e:
        print(e)

In [ ]:
# for rec in staff_assignment_records:
#     record = {}
#     record['ID_Class__c'] = rec['ID_Class__c']
#     record['staff_ID__c'] = rec['staff_ID__c']
#     record['Role__c'] = rec['Role__c']
#     if rec['StartDate__c'] != 0:
#         record['StartDate__c'] = str(rec['StartDate__c'])
#     if rec['EndDate__c'] != None:
#         record['EndDate__c'] = str(rec['EndDate__c'])
    
#     try:
#         sf.Staff_Assignment__c.create(record)
#     except Exception as e:
#         print(e)